In [1]:
#Librerias
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk

import tensorflow as tf
import os 
import numpy as np
import matplotlib.pyplot as plt
import cv2

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow_addons.layers import InstanceNormalization

In [2]:
#Para la ejecución de la GUI ejecutar todas las celdas de abajo, las primeras cargan el modelo de CycleGAN seleccionado.

In [3]:
# Weights initializer for the layers.
kernel_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)
# Gamma initializer for instance normalization.
gamma_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

class ReflectionPadding2D(layers.Layer):
    """Implements Reflection Padding as a layer.

    Args:
        padding(tuple): Amount of padding for the
        spatial dimensions.

    Returns:
        A padded tensor with the same type as the input tensor.
    """

    def __init__(self, padding=(1, 1), **kwargs):
        self.padding = tuple(padding)
        super(ReflectionPadding2D, self).__init__(**kwargs)

    def call(self, input_tensor, mask=None):
        padding_width, padding_height = self.padding
        padding_tensor = [
            [0, 0],
            [padding_height, padding_height],
            [padding_width, padding_width],
            [0, 0],
        ]
        return tf.pad(input_tensor, padding_tensor, mode="REFLECT")


def residual_block(input,activation,kernel_initializer=kernel_init,kernel_size=(3, 3),strides=(1, 1),padding="same",gamma_initializer=gamma_init,use_bias=False,):
    dim = input.shape[-1]

    # x = ReflectionPadding2D()(input_tensor)
    x = layers.Conv2D(dim,kernel_size,strides=strides,kernel_initializer=kernel_initializer,padding=padding,use_bias=use_bias,)(input)
    # x = InstanceNormalization(gamma_initializer=gamma_initializer)(x)
    x = activation(x)

    # x = ReflectionPadding2D()(x)
    x = layers.Conv2D(dim,kernel_size,strides=strides,kernel_initializer=kernel_initializer,padding=padding,use_bias=use_bias,)(x)
    # x = InstanceNormalization(gamma_initializer=gamma_initializer)(x)
    x = layers.add([x, input])
    return x


def downsample(input,filters,activation,kernel_initializer=kernel_init, kernel_size=(3, 3),strides=(2, 2), padding="same", instancenorm = True,gamma_initializer=gamma_init, use_bias=False):
    x = layers.Conv2D(filters,kernel_size,strides=strides,
        kernel_initializer=kernel_initializer, padding=padding,use_bias=use_bias)(input)
    
    if instancenorm:
        x = InstanceNormalization(gamma_initializer=gamma_initializer)(x)

    x = activation(x)
    return x


def upsample(x,filters,activation,kernel_size=(3, 3),strides=(2, 2),padding="same",kernel_initializer=kernel_init,instancenorm = True,gamma_initializer=gamma_init,use_bias=False,):
    x = layers.Conv2DTranspose(filters,kernel_size,strides=strides,padding=padding,kernel_initializer=kernel_initializer,use_bias=use_bias,)(x)
    
    if instancenorm:
        x = InstanceNormalization(gamma_initializer=gamma_initializer)(x)
    
    x = activation(x)
    return x

In [4]:
def get_resnet_generator(
    filters=64,
    num_downsampling_blocks=3,
    num_residual_blocks=9,
    num_upsample_blocks=3,
    gamma_initializer=gamma_init,
    name=None,
):
    img_input = layers.Input(shape=input_img_size, name=name + "_img_input")
    
    x1 = downsample(img_input,64,kernel_size=(7,7),strides=(1,1),activation=layers.LeakyReLU(0.2),instancenorm=False)
    x2 = downsample(x1,128,activation=layers.LeakyReLU(0.2))
    x3 = downsample(x2,256,activation=layers.LeakyReLU(0.2))
    x4 = downsample(x3,512,activation=layers.LeakyReLU(0.2))

    x = x4
    # Residual blocks
    for _ in range(num_residual_blocks):
        x = residual_block(x, activation=layers.LeakyReLU(0.2))

    # Upsampling
    # for _ in range(num_upsample_blocks):
        # filters //= 2
        # x = upsample(x, filters, activation=layers.LeakyReLU(0.2))
    
    # x_skip = layers.Concatenate(name="skip1")([x,x4])
    x = upsample(x,256,activation=layers.LeakyReLU(0.2))
    # x_skip = layers.Concatenate(name="skip2")([u1,x3])
    x = upsample(x,128,activation=layers.LeakyReLU(0.2))
    # x_skip = layers.Concatenate(name="skip3")([u2,x2])
    x = upsample(x,64,activation=layers.LeakyReLU(0.2))
    # x_skip = layers.Concatenate(name="skip4")([u3,x1])

    # Final block
    # x = ReflectionPadding2D(padding=(3, 3))(x)
    output = layers.Conv2D(3, (7, 7), padding="same",use_bias=False,activation='tanh')(x)

    model = keras.models.Model(img_input, output, name=name)
    return model

In [5]:
def get_discriminator(
    filters=64, kernel_initializer=kernel_init, num_downsampling=3, name=None
):
    img_input = layers.Input(shape=input_img_size, name=name + "_img_input")
    x = layers.Conv2D(filters,(4, 4),strides=(2, 2), padding="same", kernel_initializer=kernel_initializer,)(img_input)
    x = layers.LeakyReLU(0.2)(x)

    num_filters = filters
    for num_downsample_block in range(num_downsampling):
        num_filters *= 2
        if num_downsample_block < 2:
            x = downsample(x,filters=num_filters,activation=layers.LeakyReLU(0.2),kernel_size=(4, 4),strides=(2, 2),)
        else:
            x = downsample( x,filters=num_filters,activation=layers.LeakyReLU(0.2),kernel_size=(4, 4),strides=(1, 1),)

    x = layers.Conv2D(1, (4, 4), strides=(1, 1), padding="valid", kernel_initializer=kernel_initializer)(x)

    model = keras.models.Model(inputs=img_input, outputs=x, name=name)
    return model

In [6]:
# Define the standard image size.
orig_img_size = (286, 286)
# Size of the random crops to be used during training.
input_img_size = (128, 128, 3)

buffer_size = 32
batch_size = 1

# Get the generators
gen_G = get_resnet_generator(name="generator_G")
gen_F = get_resnet_generator(name="generator_F")

# Get the discriminators
disc_X = get_discriminator(name="discriminator_X")
disc_Y = get_discriminator(name="discriminator_Y")

c:\ProgramData\Anaconda3\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [7]:
class CycleGan(keras.Model):
    def __init__( self,generator_G,generator_F,discriminator_X,discriminator_Y,lambda_cycle=10.0,lambda_identity=0.5,):
        super(CycleGan, self).__init__()
        self.gen_G = generator_G
        self.gen_F = generator_F
        self.disc_X = discriminator_X
        self.disc_Y = discriminator_Y
        self.lambda_cycle = lambda_cycle
        self.lambda_identity = lambda_identity

    def compile( self,gen_G_optimizer,gen_F_optimizer,disc_X_optimizer,disc_Y_optimizer, gen_loss_fn, disc_loss_fn,):
        super(CycleGan, self).compile()
        self.gen_G_optimizer = gen_G_optimizer
        self.gen_F_optimizer = gen_F_optimizer
        self.disc_X_optimizer = disc_X_optimizer
        self.disc_Y_optimizer = disc_Y_optimizer
        self.generator_loss_fn = gen_loss_fn
        self.discriminator_loss_fn = disc_loss_fn
        self.cycle_loss_fn = keras.losses.MeanAbsoluteError()
        self.identity_loss_fn = keras.losses.MeanAbsoluteError()

    def train_step(self, batch_data):
        real_x, real_y = batch_data

        with tf.GradientTape(persistent=True) as tape:
            fake_y = self.gen_G(real_x, training=True)
            fake_x = self.gen_F(real_y, training=True)

            cycled_x = self.gen_F(fake_y, training=True)
            cycled_y = self.gen_G(fake_x, training=True)

            # Identity mapping
            same_x = self.gen_F(real_x, training=True)
            same_y = self.gen_G(real_y, training=True)

            # Discriminator output
            disc_real_x = self.disc_X(real_x, training=True)
            disc_fake_x = self.disc_X(fake_x, training=True)

            disc_real_y = self.disc_Y(real_y, training=True)
            disc_fake_y = self.disc_Y(fake_y, training=True)

            # Generator adverserial loss
            gen_G_loss = self.generator_loss_fn(disc_fake_y)
            gen_F_loss = self.generator_loss_fn(disc_fake_x)

            # Generator cycle loss
            cycle_loss_G = self.cycle_loss_fn(real_y, cycled_y) * self.lambda_cycle
            cycle_loss_F = self.cycle_loss_fn(real_x, cycled_x) * self.lambda_cycle

            # Generator identity loss
            id_loss_G = (
                self.identity_loss_fn(real_y, same_y)
                * self.lambda_cycle
                * self.lambda_identity
            )
            id_loss_F = (
                self.identity_loss_fn(real_x, same_x)
                * self.lambda_cycle
                * self.lambda_identity
            )

            # Total generator loss
            total_loss_G = gen_G_loss + cycle_loss_G + id_loss_G
            total_loss_F = gen_F_loss + cycle_loss_F + id_loss_F

            # Discriminator loss
            disc_X_loss = self.discriminator_loss_fn(disc_real_x, disc_fake_x)
            disc_Y_loss = self.discriminator_loss_fn(disc_real_y, disc_fake_y)

        # Get the gradients for the generators
        grads_G = tape.gradient(total_loss_G, self.gen_G.trainable_variables)
        grads_F = tape.gradient(total_loss_F, self.gen_F.trainable_variables)

        # Get the gradients for the discriminators
        disc_X_grads = tape.gradient(disc_X_loss, self.disc_X.trainable_variables)
        disc_Y_grads = tape.gradient(disc_Y_loss, self.disc_Y.trainable_variables)

        # Update the weights of the generators
        self.gen_G_optimizer.apply_gradients(
            zip(grads_G, self.gen_G.trainable_variables)
        )
        self.gen_F_optimizer.apply_gradients(
            zip(grads_F, self.gen_F.trainable_variables)
        )

        # Update the weights of the discriminators
        self.disc_X_optimizer.apply_gradients(
            zip(disc_X_grads, self.disc_X.trainable_variables)
        )
        self.disc_Y_optimizer.apply_gradients(
            zip(disc_Y_grads, self.disc_Y.trainable_variables)
        )

        return {
            "G_loss": total_loss_G,
            "F_loss": total_loss_F,
            "D_X_loss": disc_X_loss,
            "D_Y_loss": disc_Y_loss,
        }

In [8]:
# Loss function for evaluating adversarial loss
adv_loss_fn = keras.losses.MeanSquaredError()

# Define the loss function for the generators
def generator_loss_fn(fake):
    fake_loss = adv_loss_fn(tf.ones_like(fake), fake)
    return fake_loss


# Define the loss function for the discriminators
def discriminator_loss_fn(real, fake):
    real_loss = adv_loss_fn(tf.ones_like(real), real)
    fake_loss = adv_loss_fn(tf.zeros_like(fake), fake)
    return (real_loss + fake_loss) * 0.5


# Create cycle gan model
cycle_gan_model = CycleGan(
    generator_G=gen_G, generator_F=gen_F, discriminator_X=disc_X, discriminator_Y=disc_Y
)

In [9]:
#Función para cargar la imagen desde el sistema de ficheros.
def cargar_imagen():
    ruta_imagen = filedialog.askopenfilename()
    if ruta_imagen:
        imagen = Image.open(ruta_imagen)
        imagen = imagen.resize((300, 300), Image.ANTIALIAS)
        imagen2 = ImageTk.PhotoImage(imagen)
        imagen_or.config(image=imagen2)
        imagen_or.image = imagen2
        imagen_or.imagen_original = imagen

In [23]:
#Eleccion de opcion para la transformación
options = ['Transformación hombre joven a hombre adulto','Transformación hombre adulto a hombre joven','Transformacion mujer joven a mujer adulta','Transformación mujer adulta a mujer joven']
def select_combo(event):
    if select_mode.get() == options[0]:
         cycle_gan_model.load_weights('modelos/cyclegan/cycleMale')
    elif select_mode.get() == options[1] :
        cycle_gan_model.load_weights('modelos/cyclegan/cycleMale')
    elif select_mode.get() == options[2] :
         cycle_gan_model.load_weights('modelos/cyclegan/cycleFemale')
    elif select_mode.get() == options[3]:
        cycle_gan_model.load_weights('modelos/cyclegan/cycleFemale')
    else:
       print('Other is selected')

In [24]:
#Función que se encarga de la transformación
def transform_img():
    img = imagen_or.imagen_original  #Obtener imagen original
    img = img.resize((128,128))

    open_cv_image = np.array(img) 
    open_cv_image = np.expand_dims(open_cv_image, axis=0)

    if(select_mode == options[1] or select_mode == options[3]):
        prediction = cycle_gan_model.gen_F(open_cv_image)[0].numpy() #Transformar imagen original
        prediction2 = np.expand_dims(prediction,axis=0)
        recons = cycle_gan_model.gen_G(prediction2)[0].numpy() #Obtener la imagen reconstruida
    else :
        prediction = cycle_gan_model.gen_G(open_cv_image)[0].numpy() #Transformar imagen original
        prediction2 = np.expand_dims(prediction,axis=0)
        recons = cycle_gan_model.gen_F(prediction2)[0].numpy() #Obtener la imagen reconstruida


    prediction = (prediction * 127.5 + 127.5).astype(np.uint8)
    recons = (recons * 127.5 + 127.5).astype(np.uint8)

    pil_img = Image.fromarray(prediction)   #Colocar imagen transformada en la GUI
    pil_img = pil_img.resize((300,300))
    ph = ImageTk.PhotoImage(pil_img)
    imagen_tr.config(image = ph)
    imagen_tr.image = ph

    pil_recons = Image.fromarray(recons)       #Colocar imagen reconstruida en la GUI
    pil_recons = pil_recons.resize((300,300))
    ph2 = ImageTk.PhotoImage(pil_recons)
    imagen_re.config(image= ph2)
    imagen_re.image = ph2

In [25]:
#Codigo de la GUI
import tkinter as tk
import tkinter.font as tkFont
from tkinter import ttk

root = tk.Tk()
 #setting title
root.title("GUI CYCLEGAN")
#setting window size
width=1500
height=500
screenwidth = root.winfo_screenwidth()
screenheight = root.winfo_screenheight()
alignstr = '%dx%d+%d+%d' % (width, height, (screenwidth - width) / 2, (screenheight - height) / 2)
root.geometry(alignstr)
root.resizable(width=False, height=False)

load_but=tk.Button(root)
load_but["bg"] = "#e9e9ed"
ft = tkFont.Font(family='Times',size=10)
load_but["font"] = ft
load_but["fg"] = "#000000"
load_but["justify"] = "center"
load_but["text"] = "Cargar imagen"
load_but.place(x=160,y=365,width=120,height=40)
load_but["command"] = cargar_imagen

label_or = tk.Label(root)
ft = tkFont.Font(family='Times',size=12)
label_or["font"] = ft
label_or["text"] = "Imagen original "
label_or.place(x=70,y=17,width=300,height=25)
imagen_or=tk.Label(root)
imagen_or["relief"] = "solid"
imagen_or.place(x=70,y=40,width=300,height=300)

trasform_but=tk.Button(root)
trasform_but["bg"] = "#e9e9ed"
ft = tkFont.Font(family='Times',size=10)
trasform_but["font"] = ft
trasform_but["fg"] = "#000000"
trasform_but["justify"] = "center"
trasform_but["text"] = "Transformar imagen"
trasform_but.place(x=820,y=365,width=120,height=40)
trasform_but["command"] = transform_img

label_tr = tk.Label(root)
ft = tkFont.Font(family='Times',size=12)
label_tr["font"] = ft
label_tr["text"] = "Imagen transformada "
label_tr.place(x=730,y=17,width=300,height=25)
imagen_tr=tk.Label(root)
imagen_tr["relief"] = "solid"
imagen_tr.place(x=730,y=40,width=300,height=300)

label_re = tk.Label(root)
ft = tkFont.Font(family='Times',size=12)
label_re["font"] = ft
label_re["text"] = "Imagen reconstruida "
label_re.place(x=1100,y=17,width=300,height=25)
imagen_re=tk.Label(root)
imagen_re["relief"] = "solid"
imagen_re.place(x=1100,y=40,width=300,height=300)


label1 = tk.Label(root)
ft = tkFont.Font(family='Times',size=10)
label1["font"] = ft
label1["text"] = "Selecciona un modo : "
label1.place(x=400,y=60,width=300,height=25)
select_mode=ttk.Combobox(root)
select_mode['state'] = 'readonly'
select_mode['values'] = ('Transformación hombre joven a hombre adulto','Transformación hombre adulto a hombre joven','Transformacion mujer joven a mujer adulta',
                         'Transformación mujer adulta a mujer joven')
select_mode.place(x=400,y=85,width=300,height=25)
select_mode.bind('<<ComboboxSelected>>',select_combo)


root.mainloop()



Inverso
Inverso
Inverso
